In [1]:
import pandas as pd
import re
import tqdm as notebook_tqdm
from transformers import AutoModel, AutoTokenizer
from razdel import tokenize
from string import punctuation
import torch


emoj = re.compile('['
                  u'\U0001F600-\U0001F64F'
                  u'\U0001F300-\U0001F5FF'
                  u'\U0001F680-\U0001F6FF'
                  u'\U0001F1E0-\U0001F1FF'
                  u'\U00002500-\U00002BEF'
                  u'\U00002702-\U000027B0'
                  u'\U000024C2-\U0001F251'
                  u'\U0001f926-\U0001f937'
                  u'\U00010000-\U0010ffff'
                  u'\u2640-\u2642'
                  u'\u2600-\u2B55'
                  u'\u200d'
                  u'\u23cf'
                  u'\u23e9'
                  u'\u231a'
                  u'\ufe0f'
                  u'\u3030'
                  ']', re.UNICODE)
punctuation += '—'
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
vectorizer = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")
with open('../DATA/less_semantic_words.txt', 'r', encoding='utf-8') as f:
    stop_words = pd.Series(f.read().split('\n')).unique()

d:\lct-yakut-2023\apps\algorithms\LofDT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def clear_from_stopwords(text, sw):
    text = re.sub(emoj, '', text)
    tokens = list(tokenize(text))
    t = []
    for i in tokens:
        if len(i.text) > 2 and i.text not in sw:
            t.append(i.text)
    return ' '.join(t)

In [8]:
check_text = vectorizer(**tokenizer([clear_from_stopwords('с днем рождения\n 😃 Картинки и открытки ♥ vk.com/fotomagic_su ♥', stop_words)], padding=True, truncation=True, pad_to_multiple_of=65, max_length=65, return_tensors='pt'))[0].squeeze(0).sum(dim=0)

In [9]:
t = 0
d = dict()
d_l = dict()
for i in torch.load('../DATA/small_description_vectors.pt'):
    d[t] = torch.nn.CosineSimilarity(dim=1)((i.sum(dim=0)/i.shape[0]).unsqueeze(0), check_text.unsqueeze(0)).item()
    t+=1
t_l = 0
for i in torch.load('../DATA/long_description_vectors.pt'):
    d_l[t_l] = torch.nn.CosineSimilarity(dim=1)((i.sum(dim=0)/i.shape[0]).unsqueeze(0), check_text.unsqueeze(0)).item()
    t_l+=1


In [10]:
data = pd.read_excel('../DATA/Профессии_ОКСО_ ЛЦТ_Профилум.xlsx', index_col='№').loc[:, ['Отрасль текстом', 'Название профессии']]


In [11]:
data.iloc[pd.Series(d).sort_values(ascending=0).index]

,Отрасль текстом,Название профессии
№,,
36,Образование и наука,Дошкольный педагог
9,"Бизнес, менеджмент и деловые услуги",Менеджер по продажам
41,Торговля,Менеджер по работе с маркетплейсами
21,Информационные технологии (ИТ) и телекоммуникации,Frontend-разработчик
24,Информационные технологии (ИТ) и телекоммуникации,Разработчик мобильных приложений
26,Легкая промышленность,Швея-закройщица
35,Машиностроение и производство транспортных сре...,Инженер-робототехник
10,"Бизнес, менеджмент и деловые услуги",Digital-маркетолог
11,"Бизнес, менеджмент и деловые услуги",Контент-менеджер


In [7]:
data.iloc[pd.Series(d_l).sort_values(ascending=0).index]

,Отрасль текстом,Название профессии
№,,
26,Легкая промышленность,Швея-закройщица
14,Здравоохранение и медицина,Реабилитолог
8,"Бизнес, менеджмент и деловые услуги",Маркетолог
29,Машиностроение и производство транспортных сре...,Сварщик
17,Информационные технологии (ИТ) и телекоммуникации,Архитектор интеллектуальных систем управления ...
41,Торговля,Менеджер по работе с маркетплейсами
45,Транспорт и логистика,Автомеханик
43,Транспорт и логистика,Водитель
22,Информационные технологии (ИТ) и телекоммуникации,Специалист по машинному обучению
